In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
from pprint import pprint
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import re 
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [ ]:
url = 'https://docs.microsoft.com/en-us/windows-server/administration/windows-commands/windows-commands'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
commands_win = []
links = []
for li in soup.find_all('li'):
    if li.find('a'):
        a_dat = li.find('a')
        commands_win.append(a_dat.get_text())
        links.append(a_dat.get('href'))
commands_win = commands_win[8:714]
links = links[8:714]

In [ ]:
def create_par_desc(tables, url):
    tags_match = '\w+\[+:\]\<\w+\>'
    par_list = ['sub-command','option', 'paramreter','parameters', 'term', 'commands', 'subcommand', 'base parameters',
            'option flag', 'parameter']
    parameter = dict()
    if tables:
        head = tables.find_all('th')
        if head[0].text.lower() in par_list and head[1].text.lower()=='description':
            lists = tables.find_all('td')
            table = pd.read_html(url)
            tot_text = table[0][head[0].text]
            total_desc = table[0][head[1].text]
            ind = 0
            for j in range(len(tot_text)):
                if type(tot_text[j]) is str and re.findall(tags_match, tot_text[j]):
                    key_lists = []
                    info = tables.find_all('ul')[ind]
                    ind+=1
                    matches = dict(re.findall( r'(?is)<strong>\s*([^<]*?)\s*<\/strong>\s*([^<]*?)\s*<', str(info)))
                    for k, v in matches.items():
                        key_lists.append([k,v])
                    parameter[tot_text[j]] = total_desc[j].split(':')[0]+ ': ' + ', '.join('-'.join(sub) for sub in key_lists )+'. '
            
                else:
                    parameter[tot_text[j]] =  total_desc[j]
        else:
            parameter['None'] = 'None'
    else:
        parameter['None'] = 'None'
    return parameter           

In [ ]:
data = {}
data['commands'] = []
for i in range(0,len(commands_win)):
    url = 'https://docs.microsoft.com/en-us/windows-server/administration/windows-commands/'+links[i]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    desc = soup.find_all('p')#.get_text()
    #print(cmd)
    if len(desc)>1:
        first = desc[0].get_text()
        second = desc[1].get_text()
        if 'applies to: windows ' in first.lower():
            description = second
        else:
            description = first
    else:
        description = desc[0].get_text()
    extra_sections = ['warning', 'note', 'important', 'caution']
    for d in range(1,len(desc)+1):
        if desc[d-1].get_text().lower() in extra_sections:
            description = description+desc[d].get_text()
    
    syntax = soup.find('pre')
    tables = soup.find('table')
    
    if syntax:
        syntax_str = syntax.text
        syntax_data = syntax_str.split("\n")
        syntax_data = syntax_data[:-1]
        
    else:
        syntax_data = 'None'
    print(commands_win[i])
    parameter = create_par_desc(tables, url)
    
    if 'command has been deprecated' not in description.lower():
        data['commands'].append({
        'commands': commands_win[i],
        'description':description,
        'syntax':   syntax_data,
        'parameters':parameter })

In [ ]:
with open('data.txt', 'w') as outfile:
    json.dump(data, outfile)